In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r'C:\Users\boyu571\boyu571_Github\01_Kakaobank_SKKU_Research_23\data\foreign_news_data\eng_news_nocontent.csv')
df

,source,author,title,description,url,urlToImage,publishedAt,content,press
0,"{'id': None, 'name': 'Trendmicro.com'}",Trend Micro,REDBALDKNIGHT/BRONZE BUTLER’s Daserf Backdoor ...,"REDBALDKNIGHT, also known as BRONZE BUTLER and...",https://blog.trendmicro.com/trendlabs-security...,https://blog.trendmicro.com/trendlabs-security...,2017-11-07T12:34:44Z,by Joey Chen and MingYen Hsieh (Threat Analyst...,Trendmicro.com
1,"{'id': None, 'name': 'Computerworld'}","Lucas Mearian, Lucas Mearian",What is blockchain? The most disruptive tech i...,Blockchain is poised to change IT in much the ...,https://www.computerworld.com/article/3191077/...,https://images.techhive.com/images/article/201...,2017-11-08T01:06:00Z,Blockchain is poised to change IT in much the ...,Computerworld
2,"{'id': None, 'name': 'hackernoon.com'}",Yaoqi Jia,Demystifying Hashgraph: Benefits and Challenges,"Recently, on our Zilliqa social channels a num...",https://hackernoon.com/demystifying-hashgraph-...,NaN,2017-11-08T17:21:35Z,"Recently, on our Zilliqa social channels a num...",hackernoon.com
3,"{'id': None, 'name': 'Topix.com'}",NaN,BitTorrent inventor announces eco-friendly Bit...,A Bitcoin transaction wastes as much electrici...,http://www.topix.com/tech/p2p/2017/11/bittorre...,http://picproxy.topixcdn.com/pics/up-,2017-11-08T23:21:17Z,A Bitcoin transaction wastes as much electrici...,Topix.com
4,"{'id': None, 'name': 'Ml-implode.com'}",NaN,Bitcoin Fork Called Off: Prices Soar After Seg...,The price of a bitcoin went on a tear on Wedne...,http://ml-implode.com/staticnews/2017-11-09_Bi...,NaN,2017-11-09T00:00:00Z,The price of a bitcoin went on a tear on Wedne...,Ml-implode.com
...,...,...,...,...,...,...,...,...,...
744427,"{'id': None, 'name': 'Biztoc.com'}",u.today,"Negatively Correlates With Bitcoin (BTC), What...",Shiba Inu continues to focus on utility as its...,https://biztoc.com/x/8248d6342c37d743,https://c.biztoc.com/p/8248d6342c37d743/s.webp,2023-09-30T21:50:10Z,Shiba Inu continues to focus on utility as its...,Biztoc.com
744428,"{'id': None, 'name': 'Slashdot.org'}",feedfeeder,Industry leaders and policymakers weigh in on ...,Industry leaders and policymakers weigh in on ...,https://slashdot.org/firehose.pl?op=view&amp;i...,NaN,2023-09-30T21:53:20Z,Sign up for the Slashdot newsletter! OR check ...,Slashdot.org
744429,"{'id': None, 'name': 'Investing.com'}",U.Today,Shiba Inu (SHIB) Negatively Correlates With Bi...,Shiba Inu (SHIB) Negatively Correlates With Bi...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/LYNXNPE...,2023-09-30T22:00:22Z,U.Today - Dog-themed cryptocurrency is seeing ...,Investing.com
744430,"{'id': None, 'name': 'Ambcrypto.com'}",Benjamin Njiri,Can NEAR stay above $1?,"NEAR could be ripe for a price reversal, but t...",https://ambcrypto.com/?p=347645,https://statics.ambcrypto.com/wp-content/uploa...,2023-09-30T22:30:04Z,Disclaimer: The information presented does not...,Ambcrypto.com


In [327]:
df[df['press'] == 'Entrepreneur']['url'].iloc[1000]

'https://www.entrepreneur.com/article/303556'

In [329]:
Entrepreneur = df[df['press'] == 'Entrepreneur']
Entrepreneur

,source,author,title,description,url,urlToImage,publishedAt,content,press
289,"{'id': None, 'name': 'Entrepreneur'}",Yumi Clark,5 Tips for Women Entrepreneurs to Close the Sa...,Women workers make 76 cents on the dollar comp...,https://www.entrepreneur.com/article/303556,https://assets.entrepreneur.com/content/3x2/13...,2017-11-10T18:00:00Z,Ask nearly any small business owner and they'l...,Entrepreneur
331,"{'id': None, 'name': 'Entrepreneur'}",Andrew Medal,The New World of Innovation-as-a-Service,"Capturing innovation from ""the crowd"" (as in c...",https://www.entrepreneur.com/article/304483,https://assets.entrepreneur.com/content/3x2/13...,2017-11-10T20:30:00Z,Technological disruption is shattering all tho...,Entrepreneur
460,"{'id': None, 'name': 'Entrepreneur'}",Yoav Vilner,4 Emerging Fintech Trends Relevant to Every En...,Cryptocurrencies are important but there is a ...,https://www.entrepreneur.com/article/303581,https://assets.entrepreneur.com/content/3x2/13...,2017-11-11T16:00:00Z,"Be innovative, or get left behind. 2018 is fas...",Entrepreneur
918,"{'id': None, 'name': 'Entrepreneur'}",The Oracles,The Most Valuable Lessons These 6 Top Entrepre...,Hard-won wisdom that created wildly successful...,https://www.entrepreneur.com/article/304497,https://assets.entrepreneur.com/content/3x2/13...,2017-11-13T16:15:00Z,The entrepreneur’s journey is full of challeng...,Entrepreneur
1756,"{'id': None, 'name': 'Entrepreneur'}",Andrew Medal,How Blockchain Is Creating a New Future for Di...,The transparent nature of blockchain data will...,https://www.entrepreneur.com/article/304654,https://assets.entrepreneur.com/content/3x2/13...,2017-11-15T21:15:00Z,Advertising today is a multi-billion-dollar in...,Entrepreneur
...,...,...,...,...,...,...,...,...,...
624467,"{'id': None, 'name': 'Entrepreneur'}",Gleb Tsipursky,26% of U.S. Workers Would Rather Undergo a Roo...,This staggering statistic reveals a clear shif...,https://www.entrepreneur.com/growth-strategies...,https://assets.entrepreneur.com/content/3x2/20...,2023-01-27T23:00:00Z,According to a recent survey conducted by job ...,Entrepreneur
625490,"{'id': None, 'name': 'Entrepreneur'}",Steve Taplin,Why Software Talent Is Still in Demand Despite...,Despite the layoffs and fears of an economic r...,https://www.entrepreneur.com/science-technolog...,https://assets.entrepreneur.com/content/3x2/20...,2023-01-30T19:30:00Z,Talk of an economic downturn and potential rec...,Entrepreneur
625492,"{'id': None, 'name': 'Entrepreneur'}",Sam Silverman,The FBI's Most Wanted 'Crypto Queen' Scammed I...,Ruja Ignatova vanished in 2017 after orchestra...,https://www.entrepreneur.com/business-news/who...,https://assets.entrepreneur.com/content/3x2/20...,2023-01-30T19:36:00Z,"Disgraced ""Crypto Queen"" Ruja Ignatova'stime o...",Entrepreneur
625540,"{'id': None, 'name': 'Entrepreneur'}",Evgeniya Zaslavskaya,How to Build a Reputation That Will Become a R...,Here are a few tips on building a personal bra...,https://www.entrepreneur.com/growing-a-busines...,https://assets.entrepreneur.com/content/3x2/20...,2023-01-30T21:00:00Z,"As Robert Greene wrote in his book, The 48 Law...",Entrepreneur


In [282]:

url = 'https://www.entrepreneur.com/article/303556'
headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
response = requests.get(url, headers=headers)
# soup = BeautifulSoup(response.text, "html.parser")
print(response.status_code)
# soup

200


In [328]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import traceback

def get_contents(yahoo):
    contents = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for url in tqdm(yahoo['url']):
        try:
            response = requests.get(url, headers=headers, timeout=60)  # 1분 timeout 설정
            soup = BeautifulSoup(response.text, "html.parser")

            # First, try to find elements with the '.article-comm div' selector
            article_elements = soup.select('.leading-relaxed')
            
            
            # # # If not found, try the '.p-m-15' selector
            # if not article_elements:
            #     article_elements = soup.select('#fxs_article_content')

            # # If still not found, try the '.article-text' selector
            # if not article_elements:
            #     article_elements = soup.select('.p-m-15')

            if article_elements:
                content = article_elements[0].get_text()
            else:
                content = ''
            # print(article_elements)

            contents.append(content)
        except (requests.Timeout, Exception) as e:
            if isinstance(e, requests.Timeout):
                print(f"Request timed out for URL: {url}")
            else:
                print(f"Error occurred: {e}")
                traceback.print_exc()
            contents.append('')

    return contents


In [26]:
# import requests
# from bs4 import BeautifulSoup
# import re
# from tqdm import tqdm

# def get_contents(yahoo):
#     contents = []

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

#     for url in tqdm(yahoo['url']):
#         try:
#             response = requests.get(url, headers=headers)
#             soup = BeautifulSoup(response.text, "html.parser")

#             # 모든 클래스 이름 중에서 ".col-*-offset-1" 패턴을 가진 것들을 찾는다
#             classes = set()
#             for tag in soup.find_all(True, {'class': True}):
#                 for class_name in tag['class']:
#                     if re.match(r'col-\w+-offset-1', class_name):
#                         classes.add(class_name)

#             # 해당 패턴을 가진 클래스들로부터 내용을 추출한다
#             content = []
#             for class_name in classes:
#                 content.extend(soup.select(f".{class_name}"))
            
#             cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content))
#             contents.append(cleaned_content)
#         except:
#             contents.append('')

#     return contents


In [331]:
head = Entrepreneur.tail()
head['content_full'] = get_contents(head)
head

100%|██████████| 5/5 [00:56<00:00, 11.36s/it]
C:\Users\boyu571\AppData\Local\Temp\ipykernel_19872\3614095265.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  head['content_full'] = get_contents(head)


,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
624467,"{'id': None, 'name': 'Entrepreneur'}",Gleb Tsipursky,26% of U.S. Workers Would Rather Undergo a Roo...,This staggering statistic reveals a clear shif...,https://www.entrepreneur.com/growth-strategies...,https://assets.entrepreneur.com/content/3x2/20...,2023-01-27T23:00:00Z,According to a recent survey conducted by job ...,Entrepreneur,\nAccording to a recent survey conducted by jo...
625490,"{'id': None, 'name': 'Entrepreneur'}",Steve Taplin,Why Software Talent Is Still in Demand Despite...,Despite the layoffs and fears of an economic r...,https://www.entrepreneur.com/science-technolog...,https://assets.entrepreneur.com/content/3x2/20...,2023-01-30T19:30:00Z,Talk of an economic downturn and potential rec...,Entrepreneur,\nTalk of an economic downturn and potential r...
625492,"{'id': None, 'name': 'Entrepreneur'}",Sam Silverman,The FBI's Most Wanted 'Crypto Queen' Scammed I...,Ruja Ignatova vanished in 2017 after orchestra...,https://www.entrepreneur.com/business-news/who...,https://assets.entrepreneur.com/content/3x2/20...,2023-01-30T19:36:00Z,"Disgraced ""Crypto Queen"" Ruja Ignatova'stime o...",Entrepreneur,"\nDisgraced ""Crypto Queen"" Ruja Ignatova's tim..."
625540,"{'id': None, 'name': 'Entrepreneur'}",Evgeniya Zaslavskaya,How to Build a Reputation That Will Become a R...,Here are a few tips on building a personal bra...,https://www.entrepreneur.com/growing-a-busines...,https://assets.entrepreneur.com/content/3x2/20...,2023-01-30T21:00:00Z,"As Robert Greene wrote in his book, The 48 Law...",Entrepreneur,"\nAs Robert Greene wrote in his book, The 48 L..."
626079,"{'id': None, 'name': 'Entrepreneur'}",Sam Silverman,'They Don't Want to Be Left in the Past': 82% ...,"Despite the volatile nature of crypto, million...",https://www.entrepreneur.com/business-news/mil...,https://assets.entrepreneur.com/content/3x2/20...,2023-01-31T15:36:00Z,Millionaires aren't shying away from crypto de...,Entrepreneur,\nMillionaires aren't shying away from crypto ...


In [324]:
# Adding the content to the DataFrame
Entrepreneur['content_full'] = get_contents(Entrepreneur)
Entrepreneur

100%|██████████| 1251/1251 [20:16<00:00,  1.03it/s]
C:\Users\boyu571\AppData\Local\Temp\ipykernel_19872\3908206441.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  VentureBeat['content_full'] = get_contents(VentureBeat)


,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
34,"{'id': None, 'name': 'VentureBeat'}",Dean Takahashi,Nvidia steps up its transition to an AI company,Nvidia reported earnings that beat expectation...,https://venturebeat.com/2017/11/09/nvidia-step...,https://venturebeat.com/wp-content/uploads/201...,2017-11-09T21:44:32Z,Nvidia reported earnings that beat expectation...,VentureBeat,\n\n\n\n\n\n\n\n\n\n\n\nGamesBeat Next unites ...
294,"{'id': None, 'name': 'VentureBeat'}",Dean Takahashi,"Nvidia CEO: Gaming will be huge, but so will A...",Nvidia reported a stellar quarter for the thre...,https://venturebeat.com/2017/11/10/nvidia-ceo-...,https://venturebeat.com/wp-content/uploads/201...,2017-11-10T18:15:00Z,Nvidia reported a stellar quarter for the thre...,VentureBeat,\n\n\n\n\n\n\n\n\n\n\n\nGamesBeat Next unites ...
298,"{'id': None, 'name': 'VentureBeat'}",Emil Protalinski,Web Summit is the Twitter of tech conferences,I had the opportunity to attend Web Summit thi...,https://venturebeat.com/2017/11/10/web-summit-...,https://venturebeat.com/wp-content/uploads/201...,2017-11-10T18:30:21Z,I had the opportunity to attend Web Summit thi...,VentureBeat,\n\nVentureBeat presents: AI Unleashed - An ex...
2249,"{'id': None, 'name': 'VentureBeat'}",Khari Johnson,PressCoin is a cryptocurrency for investigativ...,A coalition of news organizations and journali...,https://venturebeat.com/2017/11/16/presscoin-i...,https://venturebeat.com/wp-content/uploads/201...,2017-11-17T05:21:33Z,A coalition of news organizations and journali...,VentureBeat,\n\nVentureBeat presents: AI Unleashed - An ex...
2633,"{'id': None, 'name': 'VentureBeat'}","Ian Hamilton, UploadVR",Google Blocks and Unreal helped an artist make...,Artist Jarlan Perez recently worked with Googl...,https://venturebeat.com/2017/11/18/google-bloc...,https://venturebeat.com/wp-content/uploads/201...,2017-11-18T18:25:48Z,Artist Jarlan Perez recently worked with Googl...,VentureBeat,\n\n\n\n\n\n\n\n\n\n\n\nGamesBeat Next unites ...
...,...,...,...,...,...,...,...,...,...,...
722675,"{'id': None, 'name': 'VentureBeat'}",Dean Takahashi,ZTX raises $13M for Web3 virtual world and cre...,ZTX has raised $13 million in seed funding for...,https://venturebeat.com/games/ztx-raises-13m-f...,https://venturebeat.com/wp-content/uploads/202...,2023-08-15T17:00:00Z,We're thrilled to announce the return of Games...,VentureBeat,\n\n\n\n\n\n\n\n\n\n\n\nGamesBeat Next unites ...
725921,"{'id': None, 'name': 'VentureBeat'}",Dean Takahashi,"Kunlon’s Play for Fun launches no-code, AI-bas...","Play for Fun, an indie gaming studio owned by ...",https://venturebeat.com/games/kunlons-play-for...,https://venturebeat.com/wp-content/uploads/202...,2023-08-22T11:00:00Z,We're thrilled to announce the return of Games...,VentureBeat,\n\n\n\n\n\n\n\n\n\n\n\nGamesBeat Next unites ...
733888,"{'id': None, 'name': 'VentureBeat'}",Louis Columbus,Visa’s report makes a case for outsmarting pay...,"E-commerce, retailers, and mass merchants can'...",https://venturebeat.com/security/visas-report-...,https://venturebeat.com/wp-content/uploads/202...,2023-09-07T18:43:55Z,Head over to our on-demand library to view ses...,VentureBeat,\n\nVentureBeat presents: AI Unleashed - An ex...
738980,"{'id': None, 'name': 'VentureBeat'}",Kirstie McDermott,3 great Web3 jobs to apply for this week,The global Web 3.0 market size is on the rise ...,https://venturebeat.com/programming-developmen...,https://venturebeat.com/wp-content/uploads/202...,2023-09-19T12:35:53Z,Recent analysis by Emergen Research has found ...,VentureBeat,\nRecent analysis by Emergen Research has foun...


In [285]:
head['url'].iloc[2]

'http://www.thestar.com.my/business/business-news/2017/11/13/bitcoin-tumble-erases-us$38b-as-rival-cryptocurrency-gains/'

In [323]:
head['content_full'].iloc[0]

'\n\n\n\n\n\n\n\n\n\n\n\nGamesBeat Next unites gaming industry leaders for exceptional content, networking, and deal-making opportunities. Join us on Oct 23-24 in San Francisco.\xa0 Register Now\n\n\n\n\n\n\n\n\n\n\n\n\n\nNvidia reported earnings that beat expectations and showed that the company’s focus on artificial intelligence is still paying off. For the past decade, Nvidia has been rising above graphics chips for gamers, expanding to parallel processing in data centers and lately to artificial intelligence processing for deep learning neural networks and self-driving cars.\nThe company reported earnings per share of $1.33 (up 60 percent from a year ago) on revenue of $2.6 billion (up 32 percent), beating Wall Street’s expectations.\nThe company’s stock price is up more than 100 percent in the past year on the popularity of artificial intelligence. But it slumped during the day on Thursday, along with the broader market. Analysts had expected the Santa Clara, California company to

In [325]:
empty_rows = VentureBeat[VentureBeat['content_full'] == '']
print(len(empty_rows))

5


In [308]:
empty_rows['url'].iloc[20]

'https://global.techradar.com/en-za/news/clone-trade-crypto-easily-and-safely-with-koinexpert'

<!-- # empty_rows['url'].iloc[0] -->

In [326]:
VentureBeat.to_csv(r'C:\Users\boyu571\boyu571_Github\01_Kakaobank_SKKU_Research_23\data\foreign_news_data\VentureBeat_82.csv', encoding='utf-8-sig')

In [ ]:
#article-13715 > div.post__content-wrapper > div.post-content